# Import necessary modules

In [ ]:
import pathlib

import ipywidgets as widgets

import ipyniivue
from ipyniivue import MultiplanarType, NiiVue, download_dataset

# Download required data

In [ ]:
DATA_FOLDER = pathlib.Path(ipyniivue.__file__).parent / "images"
download_dataset(
    dest_folder=DATA_FOLDER,
    files=[
        "pcasl.nii.gz",
        "aal.nii.gz",
        "mni152.nii.gz",
    ],
)

# Create NiiVue instances

In [ ]:
nv1 = NiiVue(multiplanar_force_render=True)
nv2 = NiiVue(multiplanar_force_render=True)
nv3 = NiiVue(multiplanar_force_render=True)

nv1.load_volumes([{"path": DATA_FOLDER / "pcasl.nii.gz"}])
nv2.load_volumes([{"path": DATA_FOLDER / "aal.nii.gz"}])
nv3.load_volumes([{"path": DATA_FOLDER / "mni152.nii.gz"}])

# Set up on_location_change event handlers

In [ ]:
status1 = widgets.Output()
status2 = widgets.Output()
status3 = widgets.Output()


def handle_change1(location):
    """Status 1."""
    with status1:
        status1.clear_output()
        print(location.get("string", ""))


def handle_change2(location):
    """Status 2."""
    with status2:
        status2.clear_output()
        print(location.get("string", ""))


def handle_change3(location):
    """Status 3."""
    with status3:
        status3.clear_output()
        print(location.get("string", ""))


nv1.on_location_change(handle_change1)
nv2.on_location_change(handle_change2)
nv3.on_location_change(handle_change3)

# Sync/broadcasting

In [ ]:
nv1.broadcast_to([nv2, nv3], {"2d": True, "3d": True})
nv2.broadcast_to([nv1, nv3], {"2d": True, "3d": True})
nv3.broadcast_to([nv1, nv2], {"2d": True, "3d": True})

# Create dropdown widgets for layout and synchronization options

In [ ]:
layout_options = {
    "Auto": MultiplanarType.AUTO,
    "Column": MultiplanarType.COLUMN,
    "Grid": MultiplanarType.GRID,
    "Row": MultiplanarType.ROW,
}

layout_dropdown = widgets.Dropdown(
    options=layout_options, value=MultiplanarType.AUTO, description="Layout"
)

sync_options = {"Sync Disabled": 0, "Sync 2D": 1, "Sync 3D": 2, "Sync 2D and 3D": 3}

sync_dropdown = widgets.Dropdown(options=sync_options, value=3, description="Broadcast")


def on_layout_change(change):
    """Set multiplanar layout."""
    value = change["new"]
    nv1.set_multiplanar_layout(value)
    nv2.set_multiplanar_layout(value)
    nv3.set_multiplanar_layout(value)


layout_dropdown.observe(on_layout_change, names="value")


def on_sync_change(change):
    """Update broadcast options."""
    v = change["new"]
    is2D = False
    is3D = False
    if v % 2:
        is2D = True
    if v > 1:
        is3D = True
    nv1.broadcast_to([nv2, nv3], {"2d": is2D, "3d": is3D})
    nv2.broadcast_to([nv1, nv3], {"2d": is2D, "3d": is3D})
    nv3.broadcast_to([nv1, nv2], {"2d": is2D, "3d": is3D})


sync_dropdown.observe(on_sync_change, names="value")

# Display everything

In [ ]:
# set initial values
on_layout_change({"new": layout_dropdown.value})
on_sync_change({"new": sync_dropdown.value})

# display all
display(
    widgets.VBox(
        [
            widgets.HBox([layout_dropdown, sync_dropdown]),
            widgets.VBox([nv1, nv2, nv3]),
            widgets.HBox([status1, status2, status3]),
        ]
    )
)